---
title: "Fetch analyses metadata for a study"
author:
  - name: Noah de Gunst
    affiliation:
      - id: mia
        name: Department of Computing, University of Turku, Finland
  - name: Sam Hillman
    affiliation:
      - id: mia
        name: Department of Computing, University of Turku, Finland
categories: [R]
execute: 
  eval: true
---

::: {style="max-width:1200px"}
![](./../_resources/mgnify_logo.png)

# Fetch a study using MGnifyR; download the metadata for all of its analyses

The [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1) returns data and 
relationships as JSON. [MGnifyR](https://www.bioconductor.org/packages/release/bioc/html/MGnifyR.html) 
is a package to help you read MGnify data into your R analyses.

You can find all of the other "API endpoints" using the [Browsable API interface in your web browser](https://www.ebi.ac.uk/metagenomics/api/v1).

This is an interactive code notebook (a Jupyter Notebook). To run this code, click 
into each cell and press the ▶ button in the top toolbar, or press `shift+enter`.

------------------------------------------------------------------------
:::

#### Setting the access code
First, we need to specify the accession number of the study we're working with. 
This can be done by setting the `mgnify_study_accession` variable. The accession 
number uniquely identifies the study in the MGnify database.

```{r}
#| output: false

source("./utils/variable_utils.r")

mgnify_study_accession <- get_variable_from_link_or_input('MGYS', 'Study Accession', 
                                                          'MGYS00005116')

# You can also just directly set the accession variable in code, like this:
# mgnify_study_accession <- "MGYS00005292"
```

#### Constructing a MgnifyClient object to access the database
To interact with the MGnify database, we need to create an MgnifyClient object. 
This object allows us to fetch data from MGnify, and we can configure it to use 
a cache for efficiency. 

```{r}
#| output: false
# Importing the libraries
library(vegan)
library(ggplot2)
library(mia)
library(MGnifyR)

# Check if the cache directory exists, if not, create it
if (!dir.exists("./.mgnify_cache")) {
  dir.create("./.mgnify_cache", recursive = TRUE)
}

# Create the MgnifyClient object with caching enabled
mg <- MgnifyClient(usecache = TRUE, cache_dir = "./.mgnify_cache")
```

#### Displaying the help file

```{r}
#| output: false
library(IRdisplay)
display_markdown(file = '../_resources/mgnifyr_help.md')
```

## Fetch a list of the Analyses for the Study
Using the MgnifyClient object, we can search for all analyses associated with the 
study accession number we set earlier. This will return a list of analysis accession 
numbers.

```{r}
#| output: false
analyses_accessions <- searchAnalysis(mg, "studies", mgnify_study_accession)
analyses_accessions
```

## Download metadata for the first 10 Analyses
...and put it into a dataframe.

```{r}
#| output: false
analyses_metadata_df <- getMetadata(mg, head(analyses_accessions, 10))
```

## Display metadata
The table could be big, so let's look at a sample of it (`head`).

```{r}
#| output: false
t(head(analyses_metadata_df))
```

## Download the data to a multi-assay data object

> [mia](https://microbiome.github.io/mia/) is a Bioconductor package designed to 
import, store and analyze microbiome data using an object called a `TreeSummarizedExperiment.` 
This is a tailored data container optimized for microbiome data analysis.Being 
built on the `SummarizedExperiment` class, miaverse seamlessly integrates into the 
extensive `SummarizedExperiment` ecosystem. In this example we download the MGnifyR 
data to an MAE, which contains multiple `TreeSummarizedExperiment` objects.


```{r}
#| output: false
mae <- getResult(mg, accession = analyses_accessions)
```

You use `MGnifyR` features further, for example to download data. Check the Cheat 
Sheet at the top for more.